In [1]:
from web3 import Web3
import os
from dotenv import load_dotenv
import pandas as pd
import prophet
import numpy as np
import datetime as dt
from datetime import timedelta
from prophet import Prophet
from eth_account import Account
# from web3.middleware import geth_poa_middleware
from eth_abi import decode
from eth_utils import decode_hex, to_text

import requests
import random
import json

from dune_client.client import DuneClient

import plotly.graph_objs as go

from diskcache import Cache

c:\Users\Brandyn\projects\gas_accountant\gas_accountant_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

ETHERSCAN_KEY = os.getenv("ETHERSCAN_KEY")
COINGECKO_API_KEY = os.getenv("COINGECKO_API_KEY")
DUNE_API_KEY = os.getenv('DUNE_API_KEY')
FLIPSIDE_API_KEY=os.getenv('FLIPSIDE_API_KEY')
dune = DuneClient(DUNE_API_KEY)

GAS_ACCOUNTANT = os.getenv("GAS_ACCOUNTANT")
GAS_RESERVE = os.getenv('GAS_RESERVE')
ACCOUNT_ADDRESS = os.getenv("ACCOUNT_ADDRESS")
PRIVATE_KEY = os.getenv("PRIVATE_KEY")
YIELD_FARM_ADDRESS = os.getenv("YIELD_FARM_ADDRESS")
STAKING_CONTRACT = os.getenv("STAKING_CONTRACT")
SEPOLIA_GATEWAY = os.getenv("SEPOLIA_GATEWAY")

BOT_1_ADDRESS=os.getenv("BOT_1_ADDRESS")
BOT_2_ADDRESS=os.getenv("BOT_2_ADDRESS")

In [3]:
os.chdir('..')

api = True

In [4]:
base_cache_dir = os.getcwd()
cache = Cache(os.path.join(base_cache_dir, 'cache')) 
gas_reserve_data = cache.get(f'gas_reserve_spent', pd.DataFrame())

abi_path = r'gas_accountant_contracts\contracts\artifacts'
abi_paths = []  # Assuming GAS_ACCOUNTANT_ABI_PATH is predefined

for file in os.listdir(abi_path):
    if file.endswith('.json') and "metadata" not in file:  # Exclude metadata files
        abi_paths.append(os.path.join(abi_path, file))  # Add full path

print(abi_paths)  # Debug: Check the final list

abis = {}

for path in abi_paths:
    filename = os.path.basename(path)  # Extract filename (e.g., "YieldVault.json")
    name = os.path.splitext(filename)[0]  # Remove .json extension (e.g., "YieldVault")

    with open(path, "r") as file:
        abis[name] = json.load(file)  # Use name as key

print(abis)  # Debug output

w3 = Web3(Web3.HTTPProvider(SEPOLIA_GATEWAY))

ACCOUNT = Account.from_key(PRIVATE_KEY)

w3.eth.default_account = ACCOUNT.address


['gas_accountant_contracts\\contracts\\artifacts\\BatchTransfer.json', 'gas_accountant_contracts\\contracts\\artifacts\\GasReserve.json', 'gas_accountant_contracts\\contracts\\artifacts\\IGasReserve.json', 'gas_accountant_contracts\\contracts\\artifacts\\ILiquidStaking.json', 'gas_accountant_contracts\\contracts\\artifacts\\LiquidStaking.json', 'gas_accountant_contracts\\contracts\\artifacts\\StETHStrategy.json', 'gas_accountant_contracts\\contracts\\artifacts\\TestBTC.json', 'gas_accountant_contracts\\contracts\\artifacts\\TestETH.json', 'gas_accountant_contracts\\contracts\\artifacts\\YieldVault.json']
{'BatchTransfer': {'deploy': {'VM:-': {'linkReferences': {}, 'autoDeployLib': True}, 'main:1': {'linkReferences': {}, 'autoDeployLib': True}, 'ropsten:3': {'linkReferences': {}, 'autoDeployLib': True}, 'rinkeby:4': {'linkReferences': {}, 'autoDeployLib': True}, 'kovan:42': {'linkReferences': {}, 'autoDeployLib': True}, 'goerli:5': {'linkReferences': {}, 'autoDeployLib': True}, 'Custom'

In [5]:
def dune_api_results(query_num, save_csv=False, csv_path=None):
    results = dune.get_latest_result(query_num)
    df = pd.DataFrame(results.result.rows)

    if save_csv and csv_path:
        df.to_csv(csv_path, index=False)
    return df

In [6]:
def get_token_price(token='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'):
    url = f"https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={token}&vs_currencies=usd"

    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": COINGECKO_API_KEY
    }

    response = requests.get(url, headers=headers)

    

    eth_data = response.json()

    eth_df = pd.DataFrame(eth_data)
    eth_usd = eth_df[f'{token}'].values[0]

    print(eth_usd)

    return eth_usd

In [7]:
def process_transaction(tx):
    gas_price = float(tx.get("gasPrice", np.nan))  # Allow gasPrice to be NaN
    gas_used = int(tx["gasUsed"])
    
    return {
        "blockNumber": int(tx["blockNumber"]),
        "timestamp": dt.datetime.utcfromtimestamp(int(tx["timeStamp"])),
        "transaction_hash": tx["hash"],
        "from": tx["from"],
        "to": tx["to"] if tx.get("to") else "Contract Deployment",
        "gas": int(tx["gas"]),
        "gasPrice": gas_price,  # Leave NaN if missing
        "gasUsed": gas_used,
        "tx_fee": (gas_used * gas_price / 1e18) if not np.isnan(gas_price) else np.nan,  # Handle NaN in calculation
        "contractAddress": tx.get("contractAddress"),  # Default to None if contractAddress is missing
    }


In [47]:
def get_tx_and_log_with_pagination(contract_address, start_block, end_block, etherscan_api_key,module='account',action='txlist'):
    """
    Fetch logs for a contract address with pagination support from the Etherscan API.

    Parameters:
        contract_address (str): The contract address to fetch logs for.
        start_block (int): The starting block number.
        end_block (int): The ending block number or "latest".
        etherscan_api_key (str): Your Etherscan API key.
        topic_filters (dict, optional): A dictionary of topic filters, e.g.,
            {
                "topic0": "0xe085b50dde9f45e2f6290b8f6eadc05e9f66d77b30d750cb3930c5e3430b9c1e",
                "topic1": "0x0000000000000000000000002102240d1a36a9dc9f3a4d07ee9251cb723aca89",
                "topic2": None,
                "topic3": None,
                "topic0_1_opr": "and"
            }

    Returns:
        list: A list of logs fetched from the API.
    """
    if module == 'account':
        data_pulled = 'tx'
    else:
        data_pulled = 'log'

    base_url = "https://api-sepolia.etherscan.io/api"
    logs = []  # To store all logs
    page = 1
    offset = 1000  # Max records per page

    print(f'getting fresh data')

    while True:
        # Construct the base URL
        url = (
            f"{base_url}?module={module}"
            f"&action={action}"
            f"&address={contract_address}"
            f"&fromBlock={start_block}"
            f"&toBlock={end_block}"
            f"&page={page}"
            f"&offset={offset}"
            f"&apikey={etherscan_api_key}"
        )

        try:
            # Make the API request
            response = requests.get(url)
            response.raise_for_status()

            data = response.json()
            print(data)
            if data["status"] != "1":  # Etherscan returns "1" for success
                print(f"No more {data_pulled}s or error: {data.get('message', 'Unknown error')}")
                break

            logs.extend(data["result"])  # Append the logs to the list
            print(f"Fetched {len(data['result'])} {data_pulled}s from page {page}.")

            # Stop if fewer than `offset` logs are returned
            if len(data["result"]) < offset:
                print(f"All {data_pulled}s fetched.")
                break

            page += 1  # Move to the next page

        except requests.RequestException as e:
            print(f"Error while fetching {data_pulled}s: {e}")
            break

    return logs

In [9]:
def parse_gas_log(log):
    """
    Parse a gas request log entry.

    Parameters:
        log (dict): A raw log entry from Etherscan.

    Returns:
        dict: Parsed log data.
    """
    # Extract transaction hash
    tx_hash = log["transactionHash"]

    # Extract requester's address from topic[1]
    requester = "0x" + log["topics"][1][-40:]

    # Decode data fields
    data = log["data"][2:]  # Remove "0x"
    timestamp = int(data[0:64], 16)  # Convert hex to int
    amount = int(data[64:128], 16) / 1e18  # Convert to ETH

    # Convert timestamp to human-readable format
    timestamp_human = dt.datetime.utcfromtimestamp(timestamp)

    return {
        "timestamp": timestamp_human,
        "transaction_hash": tx_hash,
        "requester": requester,
        "gas_amount_eth": amount,
    }


In [10]:
def token_prices(token_addresses, network, start_date):
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    """
    Generate a SQL query to get historical price data for given token addresses from a specific start date.

    Parameters:
    - token_addresses (list): List of token addresses.
    - start_date (str): Start date in 'YYYY-MM-DD' format.

    Returns:
    - str: The SQL query string.
    """
    # Format the addresses into the SQL VALUES clause
    addresses_clause = ", ".join(f"(LOWER('{address}'))" for address in token_addresses)

    beginning = f"'{start_date.strftime('%Y-%m-%d %H:%M:%S')}'"
    print('Beginning:', beginning)
    
    prices_query = f"""
    WITH addresses AS (
        SELECT column1 AS token_address 
        FROM (VALUES
            {addresses_clause}
        ) AS tokens(column1)
    )

    SELECT 
        hour,
        symbol,
        price
    FROM 
        {network}.price.ez_prices_hourly
    WHERE 
        token_address IN (SELECT token_address FROM addresses)
        AND hour >= DATE_TRUNC('hour', TO_TIMESTAMP({beginning}, 'YYYY-MM-DD HH24:MI:SS'))
    ORDER BY 
        hour DESC, symbol
    """

    return prices_query

In [11]:
def flipside_api_results(query, api_key, attempts=10, delay=30):
    """
    Creates and retrieves results for a query using Flipside's JSON-RPC API with pagination.

    Parameters:
    - query: str, the SQL query to execute.
    - api_key: str, your Flipside API key.
    - attempts: int, number of attempts to poll for query completion.
    - delay: int, delay (in seconds) between polling attempts.

    Returns:
    - pd.DataFrame: DataFrame containing the query results.
    """
    import requests
    import time
    import pandas as pd

    # Step 1: Create the query
    url = "https://api-v2.flipsidecrypto.xyz/json-rpc"
    headers = {
        "Content-Type": "application/json",
        "x-api-key": api_key
    }
    payload = {
        "jsonrpc": "2.0",
        "method": "createQueryRun",
        "params": [
            {
                "resultTTLHours": 1,
                "maxAgeMinutes": 0,
                "sql": query,
                "tags": {"source": "python-script", "env": "production"},
                "dataSource": "snowflake-default",
                "dataProvider": "flipside"
            }
        ],
        "id": 1
    }

    response = requests.post(url, headers=headers, json=payload)
    response_data = response.json()

    if 'error' in response_data:
        raise Exception(f"Error creating query: {response_data['error']['message']}")

    query_run_id = response_data.get('result', {}).get('queryRun', {}).get('id')
    if not query_run_id:
        raise KeyError(f"Query creation failed. Response: {response_data}")

    # Step 2: Poll for query completion
    for attempt in range(attempts):
        status_payload = {
            "jsonrpc": "2.0",
            "method": "getQueryRunResults",
            "params": [
                {
                    "queryRunId": query_run_id,
                    "format": "json",
                    "page": {"number": 1, "size": 10000}
                }
            ],
            "id": 1
        }
        response = requests.post(url, headers=headers, json=status_payload)
        resp_json = response.json()

        if 'result' in resp_json and 'rows' in resp_json['result']:
            # Step 3: Handle pagination
            all_rows = []
            page_number = 1

            while True:
                status_payload["params"][0]["page"]["number"] = page_number
                response = requests.post(url, headers=headers, json=status_payload)
                resp_json = response.json()

                if 'result' in resp_json and 'rows' in resp_json['result']:
                    rows = resp_json['result']['rows']
                    if not rows:
                        break  # No more rows to fetch
                    all_rows.extend(rows)
                    page_number += 1
                else:
                    break

            # Convert the rows to a DataFrame
            return pd.DataFrame(all_rows)

        if 'error' in resp_json and 'not yet completed' in resp_json['error'].get('message', '').lower():
            time.sleep(delay)  # Wait before retrying
        else:
            raise Exception(f"Unexpected error while fetching query results: {resp_json}")

    raise TimeoutError(f"Query did not complete after {attempts} attempts.")

In [12]:
def to_time(df):
    time_cols = ['date','dt','hour','time','day','month','year','week','timestamp','date(utc)','block_timestamp']
    for col in df.columns:
        if col.lower() in time_cols and col.lower() != 'timestamp':
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
        elif col.lower() == 'timestamp':
            df[col] = pd.to_datetime(df[col], unit='ms')
            df.set_index(col, inplace=True)
    print(df.index)
    return df 

def clean_prices(prices_df):
    print('cleaning prices')
    # Pivot the dataframe
    breakpoint()
    prices_df = prices_df.drop_duplicates(subset=['hour', 'symbol'])
    prices_df_pivot = prices_df.pivot(
        index='hour',
        columns='symbol',
        values='price'
    )
    prices_df_pivot = prices_df_pivot.reset_index()

    # Rename the columns by combining 'symbol' with a suffix
    prices_df_pivot.columns = ['dt'] + [f'{col}_price' for col in prices_df_pivot.columns[1:]]
    
    print(f'cleaned prices: {prices_df_pivot}')
    return prices_df_pivot

In [13]:
def data_processing(df,dropna=True):
    df.columns=df.columns.str.lower()
    clean_df = clean_prices(df)
    clean_df = to_time(clean_df)
    if dropna == True:
        clean_df = clean_df.dropna(axis=1, how='any')

    if '__row_index' in clean_df.columns:
        clean_df.drop(columns=['__row_index'], inplace=True)

    return clean_df

In [14]:
def update_eth_price_data(new_data):
    historical_data = cache.get(f'ETH Prices', pd.DataFrame())
    historical_data = pd.concat([historical_data, new_data]).reset_index(drop=True)
    historical_data.drop_duplicates(subset='hour', keep='last', inplace=True)
    cache.set(f'ETH Prices', historical_data)

def update_balance_data(values):
    print(f'values: {values}')
    # values = pd.DataFrame([values])
    historical_port_values = cache.get(f'balance_data')
    historical_port_values = pd.concat([historical_port_values, values]).reset_index(drop=True)
    print(f'historical_port_values: {historical_port_values}')
    # breakpoint()
    historical_port_values.drop_duplicates(subset='hour', keep='first', inplace=True)
    cache.set(f'balance_data', historical_port_values)

def update_forecast_gas_data(values):
    # print(f'values at update price: {values}')

    # Ensure the 'hour' column exists by resetting index if necessary
    if isinstance(values.index, pd.DatetimeIndex):
        values = values.reset_index().rename(columns={'index': 'hour'})
    
    if 'hour' not in values.columns:
        raise ValueError("The provided DataFrame must have a 'hour' column.")

    oracle_prices = cache.get(f'forecast_price_data', pd.DataFrame())
    # breakpoint()

    # Concatenate the new values with the existing oracle_prices
    oracle_prices = pd.concat([oracle_prices, values]).drop_duplicates(subset='hour', keep='first').reset_index(drop=True)

    oracle_prices = oracle_prices.set_index('hour')
    oracle_prices.index = pd.to_datetime(oracle_prices.index)
    oracle_prices.resample('H').ffill()

    # breakpoint()
    
    # Cache the updated oracle_prices
    cache.set(f'forecast_price_data', oracle_prices.reset_index())

    print(f'Updated forecast_price_data:\n{oracle_prices}')

def update_historical_gas_data(values):
    # print(f'values at update price: {values}')

    # Ensure the 'hour' column exists by resetting index if necessary
    if isinstance(values.index, pd.DatetimeIndex):
        values = values.reset_index().rename(columns={'index': 'hour'})
    
    if 'hour' not in values.columns:
        raise ValueError("The provided DataFrame must have a 'hour' column.")

    oracle_prices = cache.get(f'historical_gas_data', pd.DataFrame())
    # breakpoint()

    # Concatenate the new values with the existing oracle_prices
    oracle_prices = pd.concat([oracle_prices, values]).drop_duplicates(subset='hour', keep='first').reset_index(drop=True)

    # breakpoint()

    oracle_prices = oracle_prices.set_index('hour')
    oracle_prices.index = pd.to_datetime(oracle_prices.index)
    oracle_prices.resample('H').ffill()
    
    # Cache the updated oracle_prices
    cache.set(f'historical_gas_data', oracle_prices.reset_index())

    print(f'Updated historical_gas_data:\n{oracle_prices}')

def update_contract_data(data):
    print(f'new data: {data}')
    model_actions = cache.get(f'contract_data', pd.DataFrame())
    print(f'model actions before update: {data}')
    model_actions = pd.concat([model_actions, data]).reset_index(drop=True)
    model_actions.drop_duplicates(subset='hour', keep='first', inplace=True)
    cache.set(f'contract_data', model_actions)

In [15]:
def get_eth_balances(api_key, addresses):
    """
    Fetch balances for multiple Ethereum addresses from Etherscan (Sepolia).
    
    :param api_key: Your Etherscan API key as a string.
    :param addresses: A list of Ethereum addresses.
    :return: JSON response with balances.
    """
    base_url = "https://api-sepolia.etherscan.io/api"
    
    # Convert list of addresses to comma-separated string
    addresses_str = ",".join(addresses)
    
    # Construct the API request URL
    params = {
        "module": "account",
        "action": "balancemulti",
        "address": addresses_str,
        "tag": "latest",
        "apikey": api_key
    }
    
    # Make the request
    response = requests.get(base_url, params=params)
    
    # Return the JSON response
    return response.json()

# Gas Price Forecasting

In [16]:
def get_sepolia_gas_history(api, term='long_term'):
    lt_sepolia_gas_data_path = 'data/sepolia_gas_metrics.csv'
    st_sepolia_gas_data_path = 'data/current_sepolia_gas_metrics.csv'

    if api:
        if term == 'long_term':
            query = 4622627  # 2 Years worth
            sepolia_gas_data = dune_api_results(query, True, lt_sepolia_gas_data_path)
            sepolia_gas_data.to_csv(lt_sepolia_gas_data_path, index=False)  # ✅ Moved inside block
        
        elif term == 'short_term':
            query = 4650506  # Runs daily at midnight for latest prices
            sepolia_gas_data = dune_api_results(query, True, st_sepolia_gas_data_path)
            sepolia_gas_data.to_csv(st_sepolia_gas_data_path, index=False)

        else:
            print('Pass "long_term" or "short_term" as parameters')
            return None

    else:
        if term == 'long_term':
            sepolia_gas_data = pd.read_csv(lt_sepolia_gas_data_path)
        elif term == 'short_term':
            sepolia_gas_data = pd.read_csv(st_sepolia_gas_data_path)
        else:
            print('Pass "long_term" or "short_term" as parameters')
            return None

    # Convert 'hour' column to datetime format
    sepolia_gas_data['hour'] = pd.to_datetime(sepolia_gas_data['hour'])

    return sepolia_gas_data

In [17]:
import matplotlib.pyplot as plt

In [18]:
def set_global_seed(env, seed=20):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)


In [19]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def create_chart(df, columns, title, y2_col=None, tickprefix=dict(y1=None, y2=None), ticksuffix=dict(y1=None, y2=None),show_legend=False,auto_title=True,show=True,save_directory=None):
    # Create a subplot with a secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Define the color cycle
    colors = ['blue', 'red', None]  # Blue, red, then default Plotly colors
    color_index = 0  # To track the current color in the cycle

    for col in columns:
        line_color = colors[color_index]  # Get the current color from the cycle
        color_index = (color_index + 1) % len(colors)  # Update to the next color, cycling back

        if col != y2_col:
            # Add traces to primary y-axis
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df[col],
                    name=col.replace("_"," ").upper(),
                    line=dict(color=line_color) if line_color else {},
                    showlegend=show_legend
                ),
                secondary_y=False,  # Plot on primary y-axis
            )
        else:
            # Add traces to secondary y-axis
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df[y2_col],
                    name=y2_col.replace("_"," ").upper(),
                    line=dict(color=line_color) if line_color else {},
                    showlegend=show_legend
                ),
                secondary_y=True,  # Plot on secondary y-axis
            )

    # Update the layout with titles
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        legend=dict(x=0.01, y=0.99),
        template='plotly_white',
        hovermode='x unified',
        font=dict(color='black')
    )

    if auto_title:
        y1_title_text = columns[0].replace("_"," ").upper()
        y2_title_text = y2_col.replace("_"," ").upper() if y2_col else None
    else:
        y1_title_text = None
        y2_title_text = None

    # Set y-axes titles and colors
    fig.update_yaxes(title_text=y1_title_text, secondary_y=False, color='blue', tickprefix=tickprefix['y1'], ticksuffix=ticksuffix['y1'])
    fig.update_yaxes(title_text=y2_title_text, secondary_y=True, color='red', tickprefix=tickprefix['y2'], ticksuffix=ticksuffix['y2'])
    fig.update_xaxes(tickfont=dict(color='black'))

    # Show the plot
    if show:
        fig.show()

    return fig

In [20]:
from prophet import Prophet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import joblib

def evaluate_prophet_with_regressors(df, regressors, target, train_size=0.75, freq='H',model_path='prophet_model.pkl',save=False):
    """
    Trains a Prophet model with multiple regressors and evaluates its forecast.

    Parameters:
    - df: DataFrame with ['ds', 'y'] (target variable) and regressors.
    - regressors: List of column names to use as extra regressors.
    - train_size: Proportion of the dataset to use for training (default: 0.75).
    - freq: Frequency of the dataset ('H' for hourly).

    Returns:
    - Dictionary with evaluation metrics.
    """

    # Ensure correct column names
    df = df.rename(columns={'hour': 'ds', 'median_gas_price': 'y'})

    # Sort data by timestamp
    df = df.sort_values('ds')

    # Verify all regressors exist
    missing_cols = [col for col in regressors if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing regressors in dataframe: {missing_cols}")

    # Split train/test data
    split_idx = int(len(df) * train_size)
    train_df = df.iloc[:split_idx].copy()  # Training data
    test_df = df.iloc[split_idx:].copy()   # Testing data

    # Initialize Prophet model
    m = Prophet()

    # Add regressors to Prophet
    for reg in regressors:
        m.add_regressor(reg)

    # Fit model with training data
    m.fit(train_df)

    # Save model using joblib
    if save:
        joblib.dump(m, model_path)
        print(f"✅ Model saved at: {model_path}")

    # Create future dataframe
    future = m.make_future_dataframe(periods=len(test_df), freq=freq)

    # ✅ Fix: Assign correct-sized regressors to the future dataframe
    for reg in regressors:
        if reg in df.columns:
            future[reg] = df[reg].iloc[-len(future):].values
        else:
            future[reg] = 0  # Default to zero if missing

    # Predict
    forecast = m.predict(future)

    # Merge predictions with actual test values
    merged_df = test_df.merge(forecast[['ds', 'yhat']], on='ds', how='inner')

    fig = m.plot(forecast)
    plt.title(f'Forecast for {target}')
    plt.show()

    # Extract actual and predicted values
    y_true = merged_df['y'].values
    y_pred = merged_df['yhat'].values

    # Calculate evaluation metrics
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # Mean Absolute Percentage Error

    # Print results
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")

    return {
        "r2_score": r2,
        "mae": mae,
        "rmse": rmse,
        "mape": mape,
    }, merged_df


In [21]:
from itertools import product
from prophet import Prophet
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def tune_prophet_hyperparameters(target_df, train_size=0.75, forecast_horizon=24, freq="H"):
    """
    Tunes Prophet's seasonality hyperparameters using a 75/25 train-test split.

    Parameters:
    - target_df: DataFrame containing ['ds', 'y'] columns.
    - train_size: Proportion of data to use for training (default: 0.75).
    - forecast_horizon: Number of future periods to forecast (default: 24).
    - freq: Frequency of the dataset (default: 'H' for hourly data).

    Returns:
    - A dictionary with the best model, best parameters, and evaluation metrics.
    """

    # Sort data by date
    target_df = target_df.sort_values("ds")

    # Split into train (75%) and test (25%) based on `train_size`
    split_idx = int(len(target_df) * train_size)
    train_df = target_df.iloc[:split_idx]
    test_df = target_df.iloc[split_idx:]

    # Define parameter grid
    param_grid = {
        "yearly_seasonality": [True, False],
        "weekly_seasonality": [True, False],
        "daily_seasonality": [True, False],
        "seasonality_mode": ["additive", "multiplicative"]
    }

    # Generate all possible parameter combinations
    param_combinations = list(product(*param_grid.values()))

    # Initialize best model tracking
    best_mape = float("inf")
    best_params = None
    best_model = None
    best_metrics = {}

    for params in param_combinations:
        try:
            # Initialize Prophet model with given parameters
            model = Prophet(
                yearly_seasonality=params[0],
                weekly_seasonality=params[1],
                daily_seasonality=params[2],
                seasonality_mode=params[3]
            )
            model.fit(train_df)  # Fit on training data

            # Create future dataframe for the test period
            future = model.make_future_dataframe(periods=len(test_df), freq=freq)
            forecast = model.predict(future)

            # Merge forecast with actual test values
            merged_df = test_df.merge(forecast[['ds', 'yhat']], on='ds', how='inner')

            # Extract actual and predicted values
            y_true = merged_df['y'].values
            y_pred = merged_df['yhat'].values

            # Compute evaluation metrics
            r2 = r2_score(y_true, y_pred)
            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # Mean Absolute Percentage Error

            # Save best model if MAPE is lower
            if mape < best_mape:
                best_mape = mape
                best_params = params
                best_model = model
                best_metrics = {"r2": r2, "mae": mae, "rmse": rmse, "mape": mape}

        except Exception as e:
            print(f"Error with parameters {params}: {e}")

    # Print best parameters
    print("Best Parameters:", dict(zip(param_grid.keys(), best_params)))
    print(f"Best MAPE: {best_mape:.2f}%")
    print(f"Best R² Score: {best_metrics['r2']:.4f}")
    print(f"Best RMSE: {best_metrics['rmse']:.4f}")
    print(f"Best MAE: {best_metrics['mae']:.4f}")

    return {
        "best_model": best_model,
        "best_params": dict(zip(param_grid.keys(), best_params)),
        "metrics": best_metrics
    }


In [22]:
def prepare_model_dataset(long_term_sepolia_gas, target):

    target_df = long_term_sepolia_gas.set_index('hour')[[target]]

    target_df.reset_index(inplace=True)
    target_df.rename(columns={"hour":"ds","median_gas_price":"y"},inplace=True)

    target_df['ds'] = pd.to_datetime(target_df['ds'])
    print(target_df.dtypes)

    target_df['ds'] = target_df['ds'].dt.strftime('%Y-%m-%d %H:00:00')
    target_df['ds'] = pd.to_datetime(target_df['ds'])

    # Load Data
    df_gas = long_term_sepolia_gas.copy()

    # Rename timestamp column for consistency
    df_gas.rename(columns={'hour': 'ds'}, inplace=True)

    # Sort by time
    df_gas = df_gas.sort_values(by="ds")

    # Compute rolling averages
    df_gas["median_gas_price_7d"] = df_gas["median_gas_price"].rolling(window=7 * 24, min_periods=1).mean()
    df_gas["median_gas_price_30d"] = df_gas["median_gas_price"].rolling(window=30 * 24, min_periods=1).mean()

    # Drop NaN values (first few rows may have missing rolling averages)
    df_gas.dropna(inplace=True)

    # Print to check data structure
    print(df_gas.head())

    df_gas.rename(columns={target:'y'},inplace=True)
    df_gas['ds'] = pd.to_datetime(df_gas['ds']).dt.strftime('%Y-%m-%d %H:00:00')
    df_gas['ds'] = pd.to_datetime(df_gas['ds'])

    # Define regressors to use
    regressor_columns = ["max_gas_price", "min_gas_price", "p25_gas_price", "p75_gas_price", 
                        "median_gas_price_7d", "median_gas_price_30d"]

    return df_gas, regressor_columns, target_df

In [23]:
import pandas as pd
import numpy as np
from prophet import Prophet

def forecast_gas_prices(model, df, regressors, start_date, train_hours=24, forecast_hours=24):
    """
    Uses an existing Prophet model trained on the last `train_hours` of data
    and forecasts `forecast_hours + any missing hours`, starting from the last data timestamp.

    Parameters:
    - model: A fitted Prophet model.
    - df: DataFrame containing historical gas price data.
    - regressors: List of additional columns to use as extra regressors.
    - train_hours: Number of hours to use for training (default: 24).
    - forecast_hours: Number of hours to forecast (default: 24).

    Returns:
    - Forecast DataFrame with predicted hourly gas prices.
    """

    # Ensure correct column names
    df = df.rename(columns={'hour': 'ds', 'median_gas_price': 'y'})

    # Sort data by timestamp
    df = df.sort_values('ds')

    # Get the last timestamp in the dataset
    last_timestamp = df['ds'].max()

    # Get current UTC time rounded to the nearest hour
    now_utc = pd.to_datetime(start_date)

    # Calculate the gap between now and the last data point
    time_gap = (now_utc - last_timestamp).total_seconds() / 3600  # Convert to hours

    # Adjust forecast hours to account for the missing gap
    adjusted_forecast_hours = forecast_hours + int(time_gap)

    print(f"📌 Last data point: {last_timestamp}")
    print(f"📌 Time gap to now: {time_gap:.1f} hours")
    print(f"📌 Forecasting {adjusted_forecast_hours} hours from {last_timestamp}")

    # Create future dataframe starting **right after the last data point**
    future_dates = pd.date_range(start=last_timestamp + pd.Timedelta(hours=1), 
                                 periods=adjusted_forecast_hours, freq='H')

    future = pd.DataFrame({'ds': future_dates})

    # ✅ Assign regressor values for future periods
    for reg in regressors:
        if reg in df.columns:
            # Use the last known regressor value
            future[reg] = df[reg].iloc[-1]

    # Predict
    forecast = model.predict(future)

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


In [24]:
def preapre_for_live_foreast(short_term_sepolia_gas):

    forecast_sepolia = short_term_sepolia_gas.copy()

    forecast_sepolia = forecast_sepolia.sort_values(by="hour")

    # Compute rolling averages
    forecast_sepolia["median_gas_price_7d"] = forecast_sepolia["median_gas_price"].rolling(window=7 * 24, min_periods=1).mean()
    forecast_sepolia["median_gas_price_30d"] = forecast_sepolia["median_gas_price"].rolling(window=30 * 24, min_periods=1).mean()

    # Drop NaN values (first few rows may have missing rolling averages)
    forecast_sepolia.dropna(inplace=True)

    forecast_sepolia= forecast_sepolia.rename(columns={"hour":"ds","median_gas_price":"y"}).copy()

    # Print to check data structure
    print(forecast_sepolia.head())

    forecast_sepolia['ds'] = pd.to_datetime(forecast_sepolia['ds'])
    forecast_sepolia['ds'] = pd.to_datetime(forecast_sepolia['ds']
                                            ).dt.tz_localize(None)
    return forecast_sepolia

In [25]:
api = True
train_model = False
tune_hyperparameters = False

long_term_sepolia_gas = get_sepolia_gas_history(False,'long_term')
short_term_sepolia_gas = get_sepolia_gas_history(api,'short_term')

target = 'median_gas_price'
days = 30

model_path = os.path.join(base_cache_dir,'AI\prophet_model.pkl')

data_start_date = dt.datetime.now(dt.timezone.utc) - timedelta(hours=5)
data_start_date = data_start_date.strftime('%Y-%m-%d %H:00:00')

today_utc = dt.datetime.now(dt.timezone.utc) 
formatted_today_utc = today_utc.strftime('%Y-%m-%d %H:00:00')

<>:11: SyntaxWarning: invalid escape sequence '\p'
<>:11: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Brandyn\AppData\Local\Temp\ipykernel_26808\2885001202.py:11: SyntaxWarning: invalid escape sequence '\p'
  model_path = os.path.join(base_cache_dir,'AI\prophet_model.pkl')


In [26]:
df_gas, regressor_columns, target_df = prepare_model_dataset(long_term_sepolia_gas, target)

ds    datetime64[ns, UTC]
y                 float64
dtype: object
                             ds  max_gas_price  median_gas_price  \
17299 2023-04-02 00:00:00+00:00   428571428571      1.501494e+09   
17298 2023-04-02 01:00:00+00:00   428571428571      1.613547e+09   
17297 2023-04-02 02:00:00+00:00   500000000007      1.841365e+09   
17296 2023-04-02 03:00:00+00:00   300000000000      2.210777e+09   
17295 2023-04-02 04:00:00+00:00   428571428571      1.605303e+09   

       min_gas_price  p25_gas_price  p75_gas_price  median_gas_price_7d  \
17299              7   1.500000e+09   4.480613e+09         1.501494e+09   
17298              7   1.500000e+09   4.500000e+09         1.557520e+09   
17297              7   1.500000e+09   4.500000e+09         1.652135e+09   
17296              7   1.500000e+09   4.500000e+09         1.791796e+09   
17295              7   1.500000e+09   4.500000e+09         1.754497e+09   

       median_gas_price_30d  
17299          1.501494e+09  
17298         

In [27]:
if train_model:

    # Train and evaluate Prophet with regressors
    results, results_df = evaluate_prophet_with_regressors(df_gas, regressor_columns, target, train_size=0.75, freq='H',model_path=model_path,save=False)

    # Print results
    print(results)

    y_fig = create_chart(results_df.set_index('ds'),['y','yhat'],title='Actual vs Predicted',auto_title=False,show_legend=True)


In [28]:
if tune_hyperparameters:
    best_result = tune_prophet_hyperparameters(target_df.reset_index())


In [29]:
# Loading the model 

prophet_model = joblib.load(model_path)
print("✅ Model loaded successfully!")

✅ Model loaded successfully!


In [30]:
# import os

# def update_prophet_model(df, model_path=r'E:\Projects\gas_accountant\AI\prophet_model.pkl', update_threshold=24):
#     """
#     Loads an existing Prophet model, updates it if new data is available, and saves it.

#     - update_threshold: Minimum number of new hours before refitting.
#     """
#     if os.path.exists(model_path):
#         prophet_model = joblib.load(model_path)
#         last_train_ds = prophet_model.history['ds'].max()
#     else:
#         prophet_model = Prophet()
#         last_train_ds = None

#     # Check if enough new data is available
#     new_data = df[df['ds'] > last_train_ds] if last_train_ds else df

#     if len(new_data) >= update_threshold:
#         print(f"🔄 Retraining model with {len(new_data)} new data points...")
#         prophet_model.fit(df)  # Retrain with updated data
#         joblib.dump(prophet_model, model_path)
#         print("✅ Model updated and saved!")
#     else:
#         print("✅ Using existing model (No significant new data).")

#     return prophet_model


In [31]:
forecast_sepolia = preapre_for_live_foreast(short_term_sepolia_gas)

                          ds  max_gas_price             y min_gas_price  \
38 2025-03-22 00:00:00+00:00   100000000000  9.135922e+08       2722553   
37 2025-03-22 01:00:00+00:00   200000000000  9.861439e+08       2591019   
36 2025-03-22 02:00:00+00:00  1151150000000  7.773554e+08       2604430   
35 2025-03-22 03:00:00+00:00  1151150000000  1.080430e+09       2665370   
34 2025-03-22 04:00:00+00:00   100000000000  1.104320e+09       2458581   

    p25_gas_price  p75_gas_price  median_gas_price_7d  median_gas_price_30d  
38   4.675034e+06   1.503159e+09         9.135922e+08          9.135922e+08  
37   4.486747e+06   1.503124e+09         9.498681e+08          9.498681e+08  
36   4.552286e+06   1.503187e+09         8.923638e+08          8.923638e+08  
35   5.012687e+06   1.503146e+09         9.393804e+08          9.393804e+08  
34   5.028672e+06   1.503194e+09         9.723684e+08          9.723684e+08  


In [32]:
# ✅ Load or update the Prophet model
# model = update_prophet_model(forecast_sepolia, model_path=model_path, update_threshold=24)

# ✅ Forecast the next 24 hours, ensuring it starts from the **current hour**
forecast = forecast_gas_prices(prophet_model, forecast_sepolia,regressor_columns, start_date=formatted_today_utc, train_hours=24, forecast_hours=24)


📌 Last data point: 2025-03-23 14:00:00
📌 Time gap to now: 11.0 hours
📌 Forecasting 35 hours from 2025-03-23 14:00:00


C:\Users\Brandyn\AppData\Local\Temp\ipykernel_26808\64528320.py:44: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=last_timestamp + pd.Timedelta(hours=1),


In [33]:
y_fig = create_chart(forecast.set_index('ds'),['yhat'],title=f"Predicted Gas Prices <br> Through {forecast['ds'].max()}",auto_title=False,show_legend=False,show=False)
y_fig.add_shape(
    type="line",
    x0=formatted_today_utc, x1=formatted_today_utc,
    y0=min(forecast['yhat']), y1=max(forecast['yhat']),
    line=dict(color="black", width=2, dash="dash")
)

In [ ]:
# cache.clear()

2

In [34]:
update_historical_gas_data(short_term_sepolia_gas)
update_forecast_gas_data(forecast.rename(columns={'ds':'hour'}))

forecast_gas = cache.get(f'forecast_price_data')
historical_gas = cache.get(f'historical_gas_data')

Updated historical_gas_data:
                           max_gas_price  median_gas_price min_gas_price  \
hour                                                                       
2025-03-22 19:00:00+00:00   100000000000      4.702929e+08       2681125   
2025-03-22 18:00:00+00:00   208736455613      3.244647e+08       2655729   
2025-03-22 17:00:00+00:00   253400925441      6.321782e+08       2821181   
2025-03-22 16:00:00+00:00   650000000000      1.025809e+09       2780586   
2025-03-22 15:00:00+00:00  2430000000000      1.130067e+09       2793677   
...                                  ...               ...           ...   
2025-03-23 00:00:00+00:00   100000000000      9.989584e+08       2732429   
2025-03-22 23:00:00+00:00   888000000000      7.771340e+08       2533083   
2025-03-22 22:00:00+00:00   100000000000      6.788320e+08       2628586   
2025-03-22 21:00:00+00:00   190000000000      6.964049e+08       2526283   
2025-03-22 20:00:00+00:00   504290000000      7.570259e+08 

C:\Users\Brandyn\AppData\Local\Temp\ipykernel_26808\2849307527.py:64: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

C:\Users\Brandyn\AppData\Local\Temp\ipykernel_26808\2849307527.py:35: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [35]:
forecast_gas

,hour,yhat,yhat_lower,yhat_upper
0,2025-03-22 20:00:00,1.958343e+09,-4.719600e+09,8.623778e+09
1,2025-03-22 21:00:00,1.971971e+09,-4.612054e+09,8.616153e+09
2,2025-03-22 22:00:00,2.083117e+09,-5.345070e+09,8.718694e+09
3,2025-03-22 23:00:00,2.163324e+09,-4.408016e+09,8.782310e+09
4,2025-03-23 00:00:00,2.131622e+09,-4.425672e+09,9.028553e+09
5,2025-03-23 01:00:00,2.037332e+09,-4.667842e+09,8.991772e+09
6,2025-03-23 02:00:00,2.008789e+09,-4.873292e+09,9.113903e+09
7,2025-03-23 03:00:00,2.120449e+09,-4.535308e+09,8.533967e+09
8,2025-03-23 04:00:00,2.313678e+09,-4.312293e+09,8.926516e+09
9,2025-03-23 05:00:00,2.452438e+09,-3.979911e+09,9.260977e+09


In [36]:
historical_gas

,hour,max_gas_price,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price
0,2025-03-22 19:00:00+00:00,100000000000,4.702929e+08,2681125,4.935564e+06,1.503049e+09
1,2025-03-22 18:00:00+00:00,208736455613,3.244647e+08,2655729,4.753002e+06,1.503047e+09
2,2025-03-22 17:00:00+00:00,253400925441,6.321782e+08,2821181,4.950885e+06,1.503269e+09
3,2025-03-22 16:00:00+00:00,650000000000,1.025809e+09,2780586,5.246529e+06,1.503359e+09
4,2025-03-22 15:00:00+00:00,2430000000000,1.130067e+09,2793677,5.529972e+06,1.503372e+09
...,...,...,...,...,...,...
58,2025-03-23 00:00:00+00:00,100000000000,9.989584e+08,2732429,5.004870e+06,1.503431e+09
59,2025-03-22 23:00:00+00:00,888000000000,7.771340e+08,2533083,4.610498e+06,1.503029e+09
60,2025-03-22 22:00:00+00:00,100000000000,6.788320e+08,2628586,4.622533e+06,1.502693e+09
61,2025-03-22 21:00:00+00:00,190000000000,6.964049e+08,2526283,4.501470e+06,1.502486e+09


# Tracking the Test Protocol Gas Reverse

Current block number: 7655478


In [74]:
GAS_RESERVE

'0x21229F20F71882Fb61AD31Fe5501FAe26C621830'

## We can track internal TX of gas reserve contract to see how much it spends

In [76]:
# # tx_history = get_tx_with_pagination(GAS_RESERVE, current_block - 10000, current_block - 1, ETHERSCAN_KEY)
# internal_tx_history = get_tx_and_log_with_pagination(GAS_RESERVE, last_from_block, 'latest', ETHERSCAN_KEY, action='txlistinternal')

# if len(internal_tx_history) > 0:

#     processed_transactions = [process_transaction(tx) for tx in internal_tx_history]
    
#     # Convert to DataFrame
#     df = pd.DataFrame(processed_transactions)
    
#     dataset = df.copy()
#     dataset = dataset[dataset['from']==GAS_RESERVE.lower()]
#     dataset['hour'] = dataset['timestamp'].dt.strftime('%Y-%m-%d %H:00:00')
    
#     earliest_date = dataset['hour'].min()
    
#     eth_query = token_prices(['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],'ethereum',earliest_date)
#     eth_df = flipside_api_results(api_key=FLIPSIDE_API_KEY,query=eth_query)
#     eth_df['hour'] = pd.to_datetime(eth_df['hour']).dt.strftime('%Y-%m-%d %H:00:00')
#     eth_df.set_index('hour',inplace=True)
#     eth_df_wide = data_processing(eth_df.reset_index())
#     eth_df_wide.index = eth_df_wide.index.strftime('%Y-%m-%d %H:00:00')
#     update_eth_price_data(eth_df_wide)

Fetched 1 txs from page 1.
All txs fetched.
Beginning: '2025-02-07 01:00:00'
cleaning prices
cleaned prices:                     dt  WETH_price
0  2025-02-07 01:00:00     2681.46
DatetimeIndex(['2025-02-07 01:00:00'], dtype='datetime64[ns]', name='dt', freq=None)


### Here we combine gas history to get full tx history

In [ ]:
# short_term_sepolia_gas['hour'] = pd.to_datetime(short_term_sepolia_gas['hour']).dt.strftime('%Y-%m-%d %H:00:00')
# dataset_merged = dataset.merge(
#     short_term_sepolia_gas,
#     on='hour',
#     how='left'
# )
# print(dataset)

In [ ]:
# dataset_merged['hour']

0    2025-02-01 04:00:00
Name: hour, dtype: object

In [ ]:
# dataset_merged['gasPrice'] = dataset_merged['gasPrice'].fillna(dataset_merged['median_gas_price'])
# dataset_merged['tx_fee'] = dataset_merged['tx_fee'].fillna((dataset_merged['gasPrice'] * dataset_merged['gasUsed']) / 1e18)

In [ ]:
# eth_df_wide.reset_index(inplace=True)
# eth_df_wide.rename(columns={'dt':'hour'},inplace=True)

In [ ]:
# dataset_merged

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress,hour,max_gas_price,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price
0,7615059,2025-02-01 04:52:36,0xd72c0de7ae029174fe9d87e2e38d33e418272b236297...,0xf8d1f97053376a853a6856b1272be5ce1036fea1,0x3e3a83ac9dc9fd078aee45caf91626ecad6336cf,2300,NaN,55,NaN,,2025-02-01 04:00:00,NaN,NaN,NaN,NaN,NaN


In [ ]:
# eth_df_wide.columns

Index(['hour', 'WETH_price'], dtype='object')

In [ ]:
# dataset

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress,hour
0,7615059,2025-02-01 04:52:36,0xd72c0de7ae029174fe9d87e2e38d33e418272b236297...,0xf8d1f97053376a853a6856b1272be5ce1036fea1,0x3e3a83ac9dc9fd078aee45caf91626ecad6336cf,2300,NaN,55,NaN,,2025-02-01 04:00:00


In [ ]:
# dataset_merged=dataset_merged.merge(
#     eth_df_wide,
#     on='hour',
#     how='left'
# )

In [ ]:
# dataset_merged['tx_fee_usd'] = dataset_merged['tx_fee'] * dataset_merged['WETH_price']
# dataset_merged['tx_fee_usd']

0   NaN
Name: tx_fee_usd, dtype: float64

In [ ]:
# dataset_merged['transaction_hash'].values[0]

'0xd72c0de7ae029174fe9d87e2e38d33e418272b2362977db1d4623fd0b916f61a'

In [ ]:
#USE logs instead

# update_contract_data(dataset_merged)

## We can also read the event logs to see gas requested, requester, and time requested

In [ ]:
# w3.eth.block_number

7642211

In [ ]:
# logs = get_tx_and_log_with_pagination(GAS_RESERVE, w3.eth.block_number - 1000000, w3.eth.block_number, ETHERSCAN_KEY,module='logs', action='getLogs')

Fetched 1 logs from page 1.
All logs fetched.


In [ ]:
# parsed_logs = [parse_gas_log(log) for log in logs]
# for log in parsed_logs:
#     print(log)

{'timestamp': datetime.datetime(2025, 2, 1, 4, 52, 36), 'transaction_hash': '0xd72c0de7ae029174fe9d87e2e38d33e418272b2362977db1d4623fd0b916f61a', 'requester': '0x3e3a83ac9dc9fd078aee45caf91626ecad6336cf', 'gas_amount_eth': 0.0002}


In [ ]:
# df = pd.DataFrame(parsed_logs)

# # Convert timestamp to datetime format
# df["timestamp"] = pd.to_datetime(df["timestamp"])

# # Set timestamp as the index
# df.set_index("timestamp", inplace=True)

# # Sort by timestamp
# df = df.sort_index()

# # Display the first few rows
# print(df.head())

# df.reset_index(inplace=True)
# df['hour'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:00:00')

# df_merged = df.merge(
#     short_term_sepolia_gas,
#     on='hour',
#     how='left'
# )
# print(df_merged)

# df_merged=df_merged.merge(
#     eth_df_wide,
#     on='hour',
#     how='left'
# )

                                                      transaction_hash  \
timestamp                                                                
2025-02-01 04:52:36  0xd72c0de7ae029174fe9d87e2e38d33e418272b236297...   

                                                      requester  \
timestamp                                                         
2025-02-01 04:52:36  0x3e3a83ac9dc9fd078aee45caf91626ecad6336cf   

                     gas_amount_eth  
timestamp                            
2025-02-01 04:52:36          0.0002  
            timestamp                                   transaction_hash  \
0 2025-02-01 04:52:36  0xd72c0de7ae029174fe9d87e2e38d33e418272b236297...   

                                    requester  gas_amount_eth  \
0  0x3e3a83ac9dc9fd078aee45caf91626ecad6336cf          0.0002   

                  hour max_gas_price  median_gas_price min_gas_price  \
0  2025-02-01 04:00:00           NaN               NaN           NaN   

   p25_gas_price  p75_gas_price 

In [ ]:
# df_merged['address'] = GAS_RESERVE

In [ ]:
# update_contract_data(df_merged)

new data:             timestamp                                   transaction_hash  \
0 2025-02-01 04:52:36  0xd72c0de7ae029174fe9d87e2e38d33e418272b236297...   

                                    requester  gas_amount_eth  \
0  0x3e3a83ac9dc9fd078aee45caf91626ecad6336cf          0.0002   

                  hour max_gas_price  median_gas_price min_gas_price  \
0  2025-02-01 04:00:00           NaN               NaN           NaN   

   p25_gas_price  p75_gas_price  WETH_price  \
0            NaN            NaN      3312.5   

                                      address  
0  0xf8d1f97053376a853A6856B1272BE5cE1036Fea1  
model actions before update:             timestamp                                   transaction_hash  \
0 2025-02-01 04:52:36  0xd72c0de7ae029174fe9d87e2e38d33e418272b236297...   

                                    requester  gas_amount_eth  \
0  0x3e3a83ac9dc9fd078aee45caf91626ecad6336cf          0.0002   

                  hour max_gas_price  median_gas_price mi

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_20868\1263147255.py:67: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



## Here we combine gas history to get full tx history

# Tracking AI Agent Gas Costs

In [54]:
current_block = w3.eth.block_number
print(f"Current block number: {current_block}")

last_from_block = cache.get('last_from_block',None)
print(f'last_from_block: {last_from_block}')

if last_from_block is None:
    last_from_block = current_block

last_from_block

Current block number: 7967964
last_from_block: 7967829


7967829

In [55]:
last_from_block

7967829

In [58]:
addresses = [BOT_1_ADDRESS,BOT_2_ADDRESS]
addresses_lower = [addr.lower() for addr in addresses]

address_df = pd.DataFrame()

for address in addresses:
    tx_history = get_tx_and_log_with_pagination(address, last_from_block, 'latest', ETHERSCAN_KEY)

    processed_transactions = [process_transaction(tx) for tx in tx_history]

    # Convert to DataFrame
    df = pd.DataFrame(processed_transactions)

    address_df = pd.concat([address_df,df])

getting fresh data
{'status': '1', 'message': 'OK', 'result': [{'blockNumber': '7639402', 'blockHash': '0x3223987245c63b4cb284a255caa09b8aa3729544648cb12ff6c9b66306fe4153', 'timeStamp': '1738687896', 'hash': '0xa6cc6cafd2b395e3d003582c2316275960e94a9af9b70d8d8e164dbe4718271a', 'nonce': '70', 'transactionIndex': '48', 'from': '0x2102240d1a36a9dc9f3a4d07ee9251cb723aca89', 'to': '0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8', 'value': '100000000000000000', 'gas': '21000', 'gasPrice': '31638136956', 'input': '0x', 'methodId': '0x', 'functionName': '', 'contractAddress': '', 'cumulativeGasUsed': '4181998', 'txreceipt_status': '1', 'gasUsed': '21000', 'confirmations': '328626', 'isError': '0'}, {'blockNumber': '7641377', 'blockHash': '0xe0851eeb5e755c9cd99994757c41ebfd813a3f408bd99fa515e99bde96be73bd', 'timeStamp': '1738713048', 'hash': '0xa6bc3d7ccb6ee505bf1ac5aeadaab3c9946622fd3e92956a1336019cccf790d7', 'nonce': '0', 'transactionIndex': '0', 'from': '0x21efbee92e732d9d87ae7b67e0aae7a972bd23f

C:\Users\Brandyn\AppData\Local\Temp\ipykernel_26808\1092977763.py:7: DeprecationWarning:

datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).



{'status': '1', 'message': 'OK', 'result': [{'blockNumber': '7639403', 'blockHash': '0x21a9732c5fc427fd223a47080dc5a88bd735cd09dad20d6f66be5b901c8315b6', 'timeStamp': '1738687908', 'hash': '0x2ccc31eea29de5c3132fca357247148e8432a401c2f470c242eeffbf14200c42', 'nonce': '71', 'transactionIndex': '97', 'from': '0x2102240d1a36a9dc9f3a4d07ee9251cb723aca89', 'to': '0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6', 'value': '100000000000000000', 'gas': '21000', 'gasPrice': '30015195561', 'input': '0x', 'methodId': '0x', 'functionName': '', 'contractAddress': '', 'cumulativeGasUsed': '20421697', 'txreceipt_status': '1', 'gasUsed': '21000', 'confirmations': '328625', 'isError': '0'}, {'blockNumber': '7646851', 'blockHash': '0x706adaddf0f295f208832fc5c300b5a83531825f66cce35e8109a15c683b466b', 'timeStamp': '1738783656', 'hash': '0xe9fe47c9cfecc14031c68cf3a6a50837a9e6257e6e01227498f7a8827bc0e0c9', 'nonce': '0', 'transactionIndex': '27', 'from': '0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6', 'to': '0x86ce

In [59]:
cache.set('last_from_block',current_block)

True

In [60]:
address_df

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress
0,7639402,2025-02-04 16:51:36,0xa6cc6cafd2b395e3d003582c2316275960e94a9af9b7...,0x2102240d1a36a9dc9f3a4d07ee9251cb723aca89,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,21000,3.163814e+10,21000,0.000664,
1,7641377,2025-02-04 23:50:48,0xa6bc3d7ccb6ee505bf1ac5aeadaab3c9946622fd3e92...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,
2,7641863,2025-02-05 01:35:48,0xb7644d83cf9e184a7e013d314f88bffa9d76b7322957...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,
3,7641884,2025-02-05 01:40:00,0xe01a44ff892b75134227197aa96c34c226f5347f3125...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,2.000000e+10,34504,0.000690,
4,7641895,2025-02-05 01:42:36,0x39378833b2094a5c08c0973d72168c86ed9b5bbffcb4...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,
...,...,...,...,...,...,...,...,...,...,...
12,7655743,2025-02-07 03:00:12,0xd4396a1ed3bc34d12b99dc6ec080df36eee0c6c63d70...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,3.706515e+10,34504,0.001279,
13,7656607,2025-02-07 05:59:36,0x1c9f7f8abeebbedcbce1fedbf2ddce4bb915483c95bf...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,2.000000e+10,34504,0.000690,
14,7657466,2025-02-07 09:00:00,0x1cdea004325b6a368e78b9e66be97cb7a01e388ab658...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,7.136705e+10,34504,0.002462,
15,7658297,2025-02-07 11:59:48,0xf707b5e7ec637723b1ac2d80ca6ff22c0059d08d61b4...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34851,9.479326e+10,34492,0.003270,


In [61]:
addresses_lower

['0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8',
 '0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6']

In [62]:
dataset = address_df.copy()
dataset = dataset[dataset['from'].str.lower().isin(addresses_lower)]
dataset['hour'] = dataset['timestamp'].dt.strftime('%Y-%m-%d %H:00:00')

In [63]:
dataset

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress,hour
1,7641377,2025-02-04 23:50:48,0xa6bc3d7ccb6ee505bf1ac5aeadaab3c9946622fd3e92...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,,2025-02-04 23:00:00
2,7641863,2025-02-05 01:35:48,0xb7644d83cf9e184a7e013d314f88bffa9d76b7322957...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,,2025-02-05 01:00:00
3,7641884,2025-02-05 01:40:00,0xe01a44ff892b75134227197aa96c34c226f5347f3125...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,2.000000e+10,34504,0.000690,,2025-02-05 01:00:00
4,7641895,2025-02-05 01:42:36,0x39378833b2094a5c08c0973d72168c86ed9b5bbffcb4...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,,2025-02-05 01:00:00
5,7641983,2025-02-05 02:01:36,0x3a851b75c41d276d1a043a910ab5c910346c3043b76e...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,,2025-02-05 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...
12,7655743,2025-02-07 03:00:12,0xd4396a1ed3bc34d12b99dc6ec080df36eee0c6c63d70...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,3.706515e+10,34504,0.001279,,2025-02-07 03:00:00
13,7656607,2025-02-07 05:59:36,0x1c9f7f8abeebbedcbce1fedbf2ddce4bb915483c95bf...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,2.000000e+10,34504,0.000690,,2025-02-07 05:00:00
14,7657466,2025-02-07 09:00:00,0x1cdea004325b6a368e78b9e66be97cb7a01e388ab658...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,7.136705e+10,34504,0.002462,,2025-02-07 09:00:00
15,7658297,2025-02-07 11:59:48,0xf707b5e7ec637723b1ac2d80ca6ff22c0059d08d61b4...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34851,9.479326e+10,34492,0.003270,,2025-02-07 11:00:00


In [64]:
earliest_date = cache.get('earliest_date',None)

if earliest_date is None:

    earliest_date = dataset['hour'].min()

In [65]:
earliest_date

'2025-02-04 23:00:00'

In [92]:
eth_query = token_prices(['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],'ethereum',earliest_date)
eth_df = flipside_api_results(api_key='6e273210-3ddc-4380-aa6f-1c2b3e6be702	',query=eth_query)
eth_df['hour'] = pd.to_datetime(eth_df['hour']).dt.strftime('%Y-%m-%d %H:00:00')
eth_df.set_index('hour',inplace=True)
eth_df_wide = data_processing(eth_df.reset_index())
eth_df_wide.index = eth_df_wide.index.strftime('%Y-%m-%d %H:00:00')
update_eth_price_data(eth_df_wide)

Beginning: '2025-02-04 23:00:00'
cleaning prices
cleaned prices:                        dt  WETH_price
0     2025-02-04 23:00:00     2667.42
1     2025-02-05 00:00:00     2691.12
2     2025-02-05 01:00:00     2731.20
3     2025-02-05 02:00:00     2737.80
4     2025-02-05 03:00:00     2733.07
...                   ...         ...
1097  2025-03-22 16:00:00     1984.46
1098  2025-03-22 17:00:00     1985.18
1099  2025-03-22 18:00:00     1983.65
1100  2025-03-22 19:00:00     1992.78
1101  2025-03-22 20:00:00     1997.66

[1102 rows x 2 columns]
DatetimeIndex(['2025-02-04 23:00:00', '2025-02-05 00:00:00',
               '2025-02-05 01:00:00', '2025-02-05 02:00:00',
               '2025-02-05 03:00:00', '2025-02-05 04:00:00',
               '2025-02-05 05:00:00', '2025-02-05 06:00:00',
               '2025-02-05 07:00:00', '2025-02-05 08:00:00',
               ...
               '2025-03-22 11:00:00', '2025-03-22 12:00:00',
               '2025-03-22 13:00:00', '2025-03-22 14:00:00',
        

In [94]:
cache.set('earliest_date',eth_df_wide.index.max())

True

In [97]:
historical_gas

,hour,max_gas_price,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price
0,2025-01-31 23:00:00+00:00,100000000000,1.975151e+09,901905452,1.224018e+09,2.485303e+09
1,2025-01-31 22:00:00+00:00,100000000000,2.000216e+09,815791549,1.259035e+09,2.527053e+09
2,2025-01-31 21:00:00+00:00,210000000000,2.858135e+09,1193908859,2.211837e+09,3.439532e+09
3,2025-01-31 20:00:00+00:00,210000000000,3.130228e+09,1858195062,2.491308e+09,3.632018e+09
4,2025-01-31 19:00:00+00:00,500000000000,7.116758e+09,1978633021,4.255715e+09,9.735677e+09
...,...,...,...,...,...,...
137,2025-03-21 04:00:00+00:00,1151150000000,1.813647e+08,5880380,1.355540e+07,1.509049e+09
138,2025-03-21 03:00:00+00:00,252294746496,1.104580e+09,2918913,9.868163e+06,1.507118e+09
139,2025-03-21 02:00:00+00:00,100000000000,1.051033e+09,2910691,6.957304e+06,1.504383e+09
140,2025-03-21 01:00:00+00:00,200000000000,7.349040e+08,2958975,5.256760e+06,1.503441e+09


In [ ]:
historical_gas['hour'] = pd.to_datetime(historical_gas['hour']).dt.strftime('%Y-%m-%d %H:00:00')
dataset_merged = dataset.merge(
    historical_gas,
    on='hour',
    how='left'
)
print(dataset_merged

    blockNumber           timestamp  \
0       7641377 2025-02-04 23:50:48   
1       7641863 2025-02-05 01:35:48   
2       7641884 2025-02-05 01:40:00   
3       7641895 2025-02-05 01:42:36   
4       7641983 2025-02-05 02:01:36   
..          ...                 ...   
59      7654899 2025-02-06 23:59:36   
60      7655743 2025-02-07 03:00:12   
61      7656607 2025-02-07 05:59:36   
62      7657466 2025-02-07 09:00:00   
63      7658297 2025-02-07 11:59:48   

                                     transaction_hash  \
0   0xa6bc3d7ccb6ee505bf1ac5aeadaab3c9946622fd3e92...   
1   0xb7644d83cf9e184a7e013d314f88bffa9d76b7322957...   
2   0xe01a44ff892b75134227197aa96c34c226f5347f3125...   
3   0x39378833b2094a5c08c0973d72168c86ed9b5bbffcb4...   
4   0x3a851b75c41d276d1a043a910ab5c910346c3043b76e...   
..                                                ...   
59  0x61600e6f2fea437dd528598d4892908d846b3a3ff7f4...   
60  0xd4396a1ed3bc34d12b99dc6ec080df36eee0c6c63d70...   
61  0x1c9f7f8abeeb

In [100]:
dataset_merged

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress,hour,max_gas_price,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price
0,7641377,2025-02-04 23:50:48,0xa6bc3d7ccb6ee505bf1ac5aeadaab3c9946622fd3e92...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,,2025-02-04 23:00:00,2983892099615,2.032516e+09,861112041,1.207765e+09,2.533816e+09
1,7641863,2025-02-05 01:35:48,0xb7644d83cf9e184a7e013d314f88bffa9d76b7322957...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,,2025-02-05 01:00:00,NaN,NaN,NaN,NaN,NaN
2,7641884,2025-02-05 01:40:00,0xe01a44ff892b75134227197aa96c34c226f5347f3125...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,2.000000e+10,34504,0.000690,,2025-02-05 01:00:00,NaN,NaN,NaN,NaN,NaN
3,7641895,2025-02-05 01:42:36,0x39378833b2094a5c08c0973d72168c86ed9b5bbffcb4...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,,2025-02-05 01:00:00,NaN,NaN,NaN,NaN,NaN
4,7641983,2025-02-05 02:01:36,0x3a851b75c41d276d1a043a910ab5c910346c3043b76e...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,2.000000e+10,34504,0.000690,,2025-02-05 02:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,7654899,2025-02-06 23:59:36,0x61600e6f2fea437dd528598d4892908d846b3a3ff7f4...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,2.000000e+10,34504,0.000690,,2025-02-06 23:00:00,7790978309600,1.988637e+09,796984625,1.169767e+09,2.514532e+09
60,7655743,2025-02-07 03:00:12,0xd4396a1ed3bc34d12b99dc6ec080df36eee0c6c63d70...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863,3.706515e+10,34504,0.001279,,2025-02-07 03:00:00,NaN,NaN,NaN,NaN,NaN
61,7656607,2025-02-07 05:59:36,0x1c9f7f8abeebbedcbce1fedbf2ddce4bb915483c95bf...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,2.000000e+10,34504,0.000690,,2025-02-07 05:00:00,NaN,NaN,NaN,NaN,NaN
62,7657466,2025-02-07 09:00:00,0x1cdea004325b6a368e78b9e66be97cb7a01e388ab658...,0xc26204ece1f23d993200e31a5c68bd58ddd0c6b6,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863,7.136705e+10,34504,0.002462,,2025-02-07 09:00:00,NaN,NaN,NaN,NaN,NaN


In [101]:
forecast_gas['hour'] = forecast_gas['hour'].astype('str')

In [102]:
dataset_merged = dataset.merge(
    forecast_gas[['hour','yhat']],
    on='hour',
    how='left'
)
print(dataset_merged)

    blockNumber           timestamp  \
0       7641377 2025-02-04 23:50:48   
1       7641863 2025-02-05 01:35:48   
2       7641884 2025-02-05 01:40:00   
3       7641895 2025-02-05 01:42:36   
4       7641983 2025-02-05 02:01:36   
..          ...                 ...   
59      7654899 2025-02-06 23:59:36   
60      7655743 2025-02-07 03:00:12   
61      7656607 2025-02-07 05:59:36   
62      7657466 2025-02-07 09:00:00   
63      7658297 2025-02-07 11:59:48   

                                     transaction_hash  \
0   0xa6bc3d7ccb6ee505bf1ac5aeadaab3c9946622fd3e92...   
1   0xb7644d83cf9e184a7e013d314f88bffa9d76b7322957...   
2   0xe01a44ff892b75134227197aa96c34c226f5347f3125...   
3   0x39378833b2094a5c08c0973d72168c86ed9b5bbffcb4...   
4   0x3a851b75c41d276d1a043a910ab5c910346c3043b76e...   
..                                                ...   
59  0x61600e6f2fea437dd528598d4892908d846b3a3ff7f4...   
60  0xd4396a1ed3bc34d12b99dc6ec080df36eee0c6c63d70...   
61  0x1c9f7f8abeeb

In [103]:
update_contract_data(dataset_merged)

new data:     blockNumber           timestamp  \
0       7641377 2025-02-04 23:50:48   
1       7641863 2025-02-05 01:35:48   
2       7641884 2025-02-05 01:40:00   
3       7641895 2025-02-05 01:42:36   
4       7641983 2025-02-05 02:01:36   
..          ...                 ...   
59      7654899 2025-02-06 23:59:36   
60      7655743 2025-02-07 03:00:12   
61      7656607 2025-02-07 05:59:36   
62      7657466 2025-02-07 09:00:00   
63      7658297 2025-02-07 11:59:48   

                                     transaction_hash  \
0   0xa6bc3d7ccb6ee505bf1ac5aeadaab3c9946622fd3e92...   
1   0xb7644d83cf9e184a7e013d314f88bffa9d76b7322957...   
2   0xe01a44ff892b75134227197aa96c34c226f5347f3125...   
3   0x39378833b2094a5c08c0973d72168c86ed9b5bbffcb4...   
4   0x3a851b75c41d276d1a043a910ab5c910346c3043b76e...   
..                                                ...   
59  0x61600e6f2fea437dd528598d4892908d846b3a3ff7f4...   
60  0xd4396a1ed3bc34d12b99dc6ec080df36eee0c6c63d70...   
61  0x1c

# Burn Rate Calculations

In [104]:
y_fig

In [105]:
contract_data = cache.get(f'contract_data')

balance_data = cache.get(f'balance_data')

forecast_data = cache.get('forecast_price_data')

In [106]:
balance_data

,account,balance,balance_ETH,hour
0,0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89,799699418042698592,0.799699,2025-02-01 03:00:00
1,0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89,742819206854221013,0.742819,2025-02-01 05:00:00
2,0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89,245929182129654475,0.245929,2025-02-07 02:00:00


In [107]:
contract_data

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress,...,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price,WETH_price,tx_fee_usd,requester,gas_amount_eth,address,yhat
0,7612904.0,2025-01-31 21:28:24,0xaf8ab04470639668568878222952df0a4a44bc10cb6e...,0x0064c627a55aef07f4ab6fea62e516a76e38ac8a,0x0448d01970801692c22ff2eccc570f2f7cd9b889,2300.0,2.858135e+09,55.0,1.571974e-07,,...,2.858135e+09,1.193909e+09,2.211837e+09,3.439532e+09,3322.87,0.000522,NaN,NaN,NaN,NaN
1,NaN,2025-02-01 03:50:48,0xdf9aac8e3a1dbd9d194bc43bf3350c628731d84f0894...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3319.46,NaN,0x0448d01970801692c22ff2eccc570f2f7cd9b889,0.0100,NaN,NaN
2,NaN,2025-02-01 04:52:36,0xd72c0de7ae029174fe9d87e2e38d33e418272b236297...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3312.50,NaN,0x3e3a83ac9dc9fd078aee45caf91626ecad6336cf,0.0002,0xf8d1f97053376a853A6856B1272BE5cE1036Fea1,NaN
3,7641377.0,2025-02-04 23:50:48,0xa6bc3d7ccb6ee505bf1ac5aeadaab3c9946622fd3e92...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863.0,2.000000e+10,34504.0,6.900800e-04,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.431357e+09
4,7641863.0,2025-02-05 01:35:48,0xb7644d83cf9e184a7e013d314f88bffa9d76b7322957...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863.0,2.000000e+10,34504.0,6.900800e-04,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.341100e+09
5,7641983.0,2025-02-05 02:01:36,0x3a851b75c41d276d1a043a910ab5c910346c3043b76e...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863.0,2.000000e+10,34504.0,6.900800e-04,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.331598e+09
6,7642545.0,2025-02-05 03:59:48,0xdffe079a2ad92549641db2c1e72824d437576a294571...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x3fc47744b859e08c59b71c883bdc204eba100b60,34875.0,2.000000e+10,34516.0,6.903200e-04,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.462844e+09
7,7646851.0,2025-02-05 19:27:36,0x3ad7f6d2139ca3ee47ed45d7733a84e9a745671c2426...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863.0,2.367705e+10,34504.0,8.169531e-04,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.653402e+09
8,7647281.0,2025-02-05 20:59:48,0x205da71def2b3ea3933473fbba140380f316b429d62c...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x86ce29079cc2b017dbf3285fce40c911000a0f45,34863.0,2.000000e+10,34504.0,6.900800e-04,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.536851e+09
9,7647556.0,2025-02-05 21:59:36,0xb6190248c945cbcb330e06aeaa19b2234c3fca32660f...,0x21efbee92e732d9d87ae7b67e0aae7a972bd23f8,0x3fc47744b859e08c59b71c883bdc204eba100b60,34863.0,2.000000e+10,34504.0,6.900800e-04,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.559184e+09


In [108]:
burn_rate = contract_data['gas_amount_eth'].mean()

In [110]:
balance = balance_data['balance_ETH'].values[0]

In [112]:
burn_rate_ratio = abs(contract_data['gas_amount_eth'].mean()) / (balance_data['balance_ETH'].values[0] / (365*24)) 
burn_rate_ratio

np.float64(55.86599038592098)

In [113]:
sustainable_ceiling = balance_data['balance_ETH'].values[0] / (365*24)

In [114]:
# 1️⃣ Calculate average hourly ETH spent on gas
hourly_gas_spent = abs(contract_data['gas_amount_eth'].mean())

# 2️⃣ Calculate total ETH balance
eth_balance = balance_data['balance_ETH'].values[0]

# 3️⃣ Calculate ETH cash runway (hours)
eth_runway_hours = eth_balance / hourly_gas_spent

# 4️⃣ Convert to days, months, and years
eth_runway_days = eth_runway_hours / 24
eth_runway_months = eth_runway_days / 30
eth_runway_years = eth_runway_months / 12

# 5️⃣ Burn rate ratio: ETH spent per hour compared to sustainable ETH burn
burn_rate_ratio = hourly_gas_spent / (eth_balance / (365 * 24))

# ✅ Print results
print(f"ETH Burn Rate: {hourly_gas_spent}")
print(f'ETH Balance {eth_balance}')
print(f"Sustainable Ceiling: {sustainable_ceiling}")
print(f"ETH Gas Runway: {eth_runway_hours:.2f} hours")
print(f"ETH Gas Runway: {eth_runway_days:.2f} days")
print(f"ETH Gas Runway: {eth_runway_months:.2f} months")
print(f"ETH Gas Runway: {eth_runway_years:.2f} years")
print(f"ETH Burn Rate Ratio: {burn_rate_ratio:.4f}")


ETH Burn Rate: 0.0051
ETH Balance 0.7996994180426986
Sustainable Ceiling: 9.128988790441765e-05
ETH Gas Runway: 156.80 hours
ETH Gas Runway: 6.53 days
ETH Gas Runway: 0.22 months
ETH Gas Runway: 0.02 years
ETH Burn Rate Ratio: 55.8660


In [115]:
balance_data

,account,balance,balance_ETH,hour
0,0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89,799699418042698592,0.799699,2025-02-01 03:00:00
1,0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89,742819206854221013,0.742819,2025-02-01 05:00:00
2,0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89,245929182129654475,0.245929,2025-02-07 02:00:00


In [116]:
burn_rate_ratio_timeseries = {
    'date':formatted_today_utc,
    'burn_rate_ratio':burn_rate_ratio,
    'burn_rate ETH (avg hourly spend)':contract_data['gas_amount_eth'].mean(),
    'balance': balance_data['balance_ETH'].values[0]
}

In [117]:
pd.DataFrame([burn_rate_ratio_timeseries])

,date,burn_rate_ratio,burn_rate ETH (avg hourly spend),balance
0,2025-03-22 20:00:00,55.86599,0.0051,0.799699


In [137]:
dataset_merged.columns

Index(['blockNumber', 'timestamp', 'transaction_hash', 'from', 'to', 'gas',
       'gasPrice', 'gasUsed', 'tx_fee', 'contractAddress', 'hour', 'yhat'],
      dtype='object')

In [138]:
dataset_merged[['tx_fee_usd','tx_fee']]

KeyError: "['tx_fee_usd'] not in index"

In [140]:
eth_data = get_eth_balances(ETHERSCAN_KEY, [ACCOUNT_ADDRESS,BOT_1_ADDRESS,BOT_2_ADDRESS])

In [142]:
eth_data

{'status': '1',
 'message': 'OK',
 'result': [{'account': '0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89',
   'balance': '245929182129654475'},
  {'account': '0x21efbeE92E732D9d87Ae7b67E0aae7a972bd23F8',
   'balance': '28898211754361431'},
  {'account': '0xc26204ecE1f23d993200E31A5C68bD58DDd0c6b6',
   'balance': '84182291127744928'}]}

In [143]:
pd.DataFrame(eth_data['result'])

,account,balance
0,0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89,245929182129654475
1,0x21efbeE92E732D9d87Ae7b67E0aae7a972bd23F8,28898211754361431
2,0xc26204ecE1f23d993200E31A5C68bD58DDd0c6b6,84182291127744928


In [146]:
current_balances = pd.DataFrame(eth_data['result'])
current_balances['balance'] = pd.to_numeric(current_balances['balance'], errors='coerce')

# Convert from wei to ETH
current_balances['balance_ETH'] = current_balances['balance'] / 1e18

In [147]:
current_balances['hour'] = formatted_today_utc

In [148]:
current_balances

,account,balance,balance_ETH,hour
0,0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89,245929182129654475,0.245929,2025-02-07 02:00:00
1,0x21efbeE92E732D9d87Ae7b67E0aae7a972bd23F8,28898211754361431,0.028898,2025-02-07 02:00:00
2,0xc26204ecE1f23d993200E31A5C68bD58DDd0c6b6,84182291127744928,0.084182,2025-02-07 02:00:00


In [170]:
update_balance_data(current_balances)

values:                                       account             balance  \
0  0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89  245929182129654475   
1  0x21efbeE92E732D9d87Ae7b67E0aae7a972bd23F8   28898211754361431   
2  0xc26204ecE1f23d993200E31A5C68bD58DDd0c6b6   84182291127744928   

   balance_ETH                 hour  
0     0.245929  2025-02-07 02:00:00  
1     0.028898  2025-02-07 02:00:00  
2     0.084182  2025-02-07 02:00:00  
historical_port_values:                                       account             balance  \
0  0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89  799699418042698592   
1  0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89  742819206854221013   
2  0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89  245929182129654475   
3  0x2102240d1A36a9DC9F3A4d07eE9251cb723ACa89  245929182129654475   
4  0x21efbeE92E732D9d87Ae7b67E0aae7a972bd23F8   28898211754361431   
5  0xc26204ecE1f23d993200E31A5C68bD58DDd0c6b6   84182291127744928   

   balance_ETH                 hour  
0     0.799699  2

In [150]:
len(forecast_data.index)

149

In [151]:
len(forecast_data[forecast_data['hour'] >= formatted_today_utc].index)

25

In [152]:
hourly_burn_wei = contract_data['gas_amount_eth'].mean() * 1e18
hourly_burn_wei


5100000000000000.0

In [153]:
contract_data["gas_amount_eth"] * 1e18

0             NaN
1    1.000000e+16
2    2.000000e+14
Name: gas_amount_eth, dtype: float64

In [154]:
contract_data["rolling_burn_wei"] = (
    contract_data["gas_amount_eth"] * 1e18
).rolling(window=24, min_periods=1).mean()

In [155]:
forecast_period = forecast_data[forecast_data['hour'] >= formatted_today_utc]

In [156]:
# Extend rolling average trend for the forecast period
future_burn_trend = (
    contract_data["rolling_burn_wei"].iloc[-24:].mean()
)  # Use the last 24-hour average as a baseline

# Apply this rolling average trend to future timestamps
estimated_rolling_burn_df = pd.DataFrame(
    index=forecast_period["hour"],
    data=[future_burn_trend] * len(forecast_period),
    columns=["estimated_burn_wei"],
)


In [157]:
estimated_rolling_burn_df

,estimated_burn_wei
hour,
2025-02-07 02:00:00,7.550000e+15
2025-02-07 03:00:00,7.550000e+15
2025-02-07 04:00:00,7.550000e+15
2025-02-07 05:00:00,7.550000e+15
2025-02-07 06:00:00,7.550000e+15
2025-02-07 07:00:00,7.550000e+15
2025-02-07 08:00:00,7.550000e+15
2025-02-07 09:00:00,7.550000e+15
2025-02-07 10:00:00,7.550000e+15


In [ ]:
estimated_static_future_burn = pd.DataFrame(index=forecast_data[forecast_data['hour'] >= formatted_today_utc]['hour'],data=future_burn_trend,columns=['estimated burn'])
estimated_static_future_burn

,estimated burn
hour,
2025-02-01 05:00:00,7.550000e+15
2025-02-01 06:00:00,7.550000e+15
2025-02-01 07:00:00,7.550000e+15
2025-02-01 08:00:00,7.550000e+15
2025-02-01 09:00:00,7.550000e+15
2025-02-01 10:00:00,7.550000e+15
2025-02-01 11:00:00,7.550000e+15
2025-02-01 12:00:00,7.550000e+15
2025-02-01 13:00:00,7.550000e+15


In [ ]:
# estimated_burn_df = pd.DataFrame(index=forecast_period["hour"])

# # Assign last N rolling values, allowing for missing values
# # estimated_burn_df["estimated_burn_wei"] = contract_data["rolling_burn_wei"].iloc[-len(forecast_period):].values

# # Interpolate missing values smoothly
# estimated_burn_df["estimated_burn_wei"] = contract_data["rolling_burn_wei"].interpolate(method="linear")


In [ ]:
estimated_rolling_burn_df

,estimated_burn_wei
hour,
2025-02-01 05:00:00,7.550000e+15
2025-02-01 06:00:00,7.550000e+15
2025-02-01 07:00:00,7.550000e+15
2025-02-01 08:00:00,7.550000e+15
2025-02-01 09:00:00,7.550000e+15
2025-02-01 10:00:00,7.550000e+15
2025-02-01 11:00:00,7.550000e+15
2025-02-01 12:00:00,7.550000e+15
2025-02-01 13:00:00,7.550000e+15


In [ ]:
# ✅ Convert gas price from Gwei to Wei
forecasted_costs = estimated_rolling_burn_df.merge(
    forecast_data[["hour", "yhat"]], on="hour", how="left"
)

# ✅ Convert `yhat` from Gwei to Wei before multiplying
forecasted_costs["yhat_wei"] = forecasted_costs["yhat"] * 1e9  # Convert Gwei → Wei

# ✅ Calculate estimated gas cost in ETH (Burn * Gas Price in Wei)
forecasted_costs["estimated_gas_cost_eth"] = (
    forecasted_costs["estimated_burn_wei"] * forecasted_costs["yhat_wei"]
) / 1e18  # Convert back to ETH


In [ ]:
# ✅ Convert `yhat` from Gwei → ETH
forecasted_costs["yhat_eth"] = forecasted_costs["yhat"] / 1e9  # Correct conversion

# ✅ Compute estimated gas cost in ETH
forecasted_costs["estimated_gas_cost_eth"] = (
    (forecasted_costs["estimated_burn_wei"] / 1e18) * forecasted_costs["yhat_eth"]
)
forecasted_costs["estimated_gas_cost_eth"]

0     0.026549
1     0.026643
2     0.026162
3     0.026107
4     0.027004
5     0.028301
6     0.028853
7     0.028025
8     0.026333
9     0.024988
10    0.024778
11    0.025364
12    0.025677
13    0.025000
14    0.023670
15    0.022714
16    0.022816
17    0.023656
18    0.024261
19    0.024022
20    0.023310
21    0.023094
22    0.023937
23    0.025396
24    0.026444
Name: estimated_gas_cost_eth, dtype: float64

In [ ]:
forecasted_costs["yhat_eth"]

0     3.516379
1     3.528907
2     3.465126
3     3.457907
4     3.576623
5     3.748414
6     3.821640
7     3.711959
8     3.487767
9     3.309694
10    3.281897
11    3.359492
12    3.400893
13    3.311264
14    3.135069
15    3.008421
16    3.022050
17    3.133195
18    3.213402
19    3.181701
20    3.087411
21    3.058868
22    3.170527
23    3.363756
24    3.502517
Name: yhat_eth, dtype: float64

In [ ]:
forecasted_costs

,hour,estimated_burn_wei,yhat,yhat_wei,estimated_gas_cost_eth
0,2025-02-01 05:00:00,7.550000e+15,3.516379e+09,3.516379e+18,2.654866e+16
1,2025-02-01 06:00:00,7.550000e+15,3.528907e+09,3.528907e+18,2.664325e+16
2,2025-02-01 07:00:00,7.550000e+15,3.465126e+09,3.465126e+18,2.616170e+16
3,2025-02-01 08:00:00,7.550000e+15,3.457907e+09,3.457907e+18,2.610720e+16
4,2025-02-01 09:00:00,7.550000e+15,3.576623e+09,3.576623e+18,2.700350e+16
5,2025-02-01 10:00:00,7.550000e+15,3.748414e+09,3.748414e+18,2.830052e+16
6,2025-02-01 11:00:00,7.550000e+15,3.821640e+09,3.821640e+18,2.885338e+16
7,2025-02-01 12:00:00,7.550000e+15,3.711959e+09,3.711959e+18,2.802529e+16
8,2025-02-01 13:00:00,7.550000e+15,3.487767e+09,3.487767e+18,2.633264e+16
9,2025-02-01 14:00:00,7.550000e+15,3.309694e+09,3.309694e+18,2.498819e+16


In [ ]:
forecasted_costs["yhat"]

0     3.516379e+09
1     3.528907e+09
2     3.465126e+09
3     3.457907e+09
4     3.576623e+09
5     3.748414e+09
6     3.821640e+09
7     3.711959e+09
8     3.487767e+09
9     3.309694e+09
10    3.281897e+09
11    3.359492e+09
12    3.400893e+09
13    3.311264e+09
14    3.135069e+09
15    3.008421e+09
16    3.022050e+09
17    3.133195e+09
18    3.213402e+09
19    3.181701e+09
20    3.087411e+09
21    3.058868e+09
22    3.170527e+09
23    3.363756e+09
24    3.502517e+09
Name: yhat, dtype: float64

In [ ]:
forecasted_costs["estimated_burn_wei"] / 1e18

0     0.00755
1     0.00755
2     0.00755
3     0.00755
4     0.00755
5     0.00755
6     0.00755
7     0.00755
8     0.00755
9     0.00755
10    0.00755
11    0.00755
12    0.00755
13    0.00755
14    0.00755
15    0.00755
16    0.00755
17    0.00755
18    0.00755
19    0.00755
20    0.00755
21    0.00755
22    0.00755
23    0.00755
24    0.00755
Name: estimated_burn_wei, dtype: float64

In [ ]:
forecasted_costs["estimated_gas_cost_eth"]

0     2.654866e+07
1     2.664325e+07
2     2.616170e+07
3     2.610720e+07
4     2.700350e+07
5     2.830052e+07
6     2.885338e+07
7     2.802529e+07
8     2.633264e+07
9     2.498819e+07
10    2.477832e+07
11    2.536416e+07
12    2.567674e+07
13    2.500005e+07
14    2.366977e+07
15    2.271358e+07
16    2.281648e+07
17    2.365562e+07
18    2.426119e+07
19    2.402184e+07
20    2.330995e+07
21    2.309445e+07
22    2.393748e+07
23    2.539636e+07
24    2.644400e+07
Name: estimated_gas_cost_eth, dtype: float64

In [ ]:
forecasted_costs["estimated_burn_wei"]

0     7.550000e+15
1     7.550000e+15
2     7.550000e+15
3     7.550000e+15
4     7.550000e+15
5     7.550000e+15
6     7.550000e+15
7     7.550000e+15
8     7.550000e+15
9     7.550000e+15
10    7.550000e+15
11    7.550000e+15
12    7.550000e+15
13    7.550000e+15
14    7.550000e+15
15    7.550000e+15
16    7.550000e+15
17    7.550000e+15
18    7.550000e+15
19    7.550000e+15
20    7.550000e+15
21    7.550000e+15
22    7.550000e+15
23    7.550000e+15
24    7.550000e+15
Name: estimated_burn_wei, dtype: float64

In [ ]:
forecasted_costs["estimated_gas_cost_eth"]

0     2.654866e+07
1     2.664325e+07
2     2.616170e+07
3     2.610720e+07
4     2.700350e+07
5     2.830052e+07
6     2.885338e+07
7     2.802529e+07
8     2.633264e+07
9     2.498819e+07
10    2.477832e+07
11    2.536416e+07
12    2.567674e+07
13    2.500005e+07
14    2.366977e+07
15    2.271358e+07
16    2.281648e+07
17    2.365562e+07
18    2.426119e+07
19    2.402184e+07
20    2.330995e+07
21    2.309445e+07
22    2.393748e+07
23    2.539636e+07
24    2.644400e+07
Name: estimated_gas_cost_eth, dtype: float64

In [ ]:
create_chart(forecasted_costs.set_index('hour'),title='forecasted contract costs',columns=['estimated_gas_cost_eth'])

# Gas Reserve

In [ ]:
forecasted_costs

In [ ]:
forecast_data[]

In [ ]:
gas_reserve_logs["gas_amount_wei"] = gas_reserve_logs["gas_amount_eth"] * 1e18


In [ ]:
df_merged['hour'] = pd.to_datetime(df_merged['hour'])

In [ ]:
# gas_burn_data = df_merged.merge(forecast_data, on="hour", how="left")

# # Estimate ETH spent using forecasted gas prices
# gas_burn_data["estimated_burn_eth"] = gas_burn_data["gas_amount_eth"] * gas_burn_data["yhat"]

In [100]:
# os.chdir('notebooks')